In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

def calculate_jaccard_index(arrays):
    """
    Calculates the Jaccard index for pairs of overlapping time windows.
    
    Parameters:
    arrays (list of np.array): A list of 1D arrays where each array represents time windows.
    
    Returns:
    pd.DataFrame: A DataFrame containing pairs of window ids and their Jaccard index.
    """
    # Concatenate all arrays to identify unique window IDs and the total length
    concatenated = np.concatenate(arrays)
    unique_windows = np.unique(concatenated)
    unique_windows = unique_windows[unique_windows > 0]  # Exclude 0, as it represents no window
    
    # Create a sparse matrix of shape (num_unique_windows, total_length)
    total_length = len(arrays[0])
    num_windows = len(unique_windows)
    window_to_index = {window: idx for idx, window in enumerate(unique_windows)}
    
    data = []
    rows = []
    cols = []
    
    for array in arrays:
        for time_point, window in enumerate(array):
            if window > 0:
                rows.append(window_to_index[window])
                cols.append(time_point)
                data.append(1)
    
    sparse_matrix = csr_matrix((data, (rows, cols)), shape=(num_windows, total_length))
    
    jaccard_results = []
    
    # Compare each pair of unique windows
    for i in range(num_windows):
        for j in range(i + 1, num_windows):
            # Find intersection and union
            intersection = sparse_matrix[i].multiply(sparse_matrix[j]).sum()
            union = sparse_matrix[i].maximum(sparse_matrix[j]).sum()
            
            if union > 0:
                jaccard_index = intersection / union
                if intersection > 0:  # Only consider overlapping windows
                    window1 = unique_windows[i]
                    window2 = unique_windows[j]
                    jaccard_results.append((window1, window2, jaccard_index))
    
    # Convert results to a DataFrame
    jaccard_df = pd.DataFrame(jaccard_results, columns=['Window1', 'Window2', 'JaccardIndex'])
    
    return jaccard_df

# Example usage
array1 = np.array([0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 2, 2, 2, 0, 0])
array2 = np.array([3, 3, 0, 0, 0, 4, 4, 0, 0, 0, 0, 5, 5, 5, 5])
array3 = np.array([0, 0, 0, 0, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0])

results_df = calculate_jaccard_index([array1, array2, array3])
print(results_df)

# Example filtering
filtered_results_df = results_df[results_df['JaccardIndex'] > 0.5]
print(filtered_results_df)


   Window1  Window2  JaccardIndex
0        1        4      0.666667
1        1        6      1.000000
2        2        5      0.333333
3        4        6      0.666667
   Window1  Window2  JaccardIndex
0        1        4      0.666667
1        1        6      1.000000
3        4        6      0.666667


In [5]:
slice1 = slice(4, 7, 1)
slice2 = slice(9, 13, 1)
slice3 = slice(0, 2, 1)
slice4 = slice(5, 7, 1)
slice5 = slice(11, 15, 1)
slice6 = slice(4, 8, 1)

In [10]:
def calculate_jaccard_index(slice1, slice2):
    range1 = set(range(slice1.start, slice1.stop))
    range2 = set(range(slice2.start, slice2.stop))
    intersection = len(range1.intersection(range2))
    union = len(range1.union(range2))
    return intersection / union if union else 0

def merge_slices(slice1, slice2):
    new_start = min(slice1.start, slice2.start)
    new_stop = max(slice1.stop, slice2.stop)
    return slice(new_start, new_stop)

def can_merge(slice1, slice2, threshold):
    return calculate_jaccard_index(slice1, slice2) > threshold

def merge_slices_if_possible(slices, threshold):
    merged = []
    while slices:
        current = slices.pop(0)
        for i, other in enumerate(slices):
            if can_merge(current, other, threshold):
                current = merge_slices(current, other)
                slices.pop(i)
                slices.insert(0, current)  # Re-insert at start to check new slice against all others again
                break
        else:  # No merge occurred
            merged.append(current)
    return merged

def get_merged_slices(slices, threshold):
    return merge_slices_if_possible(slices, threshold)

# Example usage
slices = [slice(4, 7, 1), slice(9, 13, 1), slice(0, 2, 1), slice(5, 7, 1), slice(11, 15, 1), slice(4, 8, 1)]
threshold = 0.5
merged_slices = get_merged_slices(slices, threshold)
print(merged_slices)

[slice(4, 8, None), slice(9, 13, 1), slice(0, 2, 1), slice(11, 15, 1)]


In [ ]:
slice1 = slice(4, 7, 1) #
slice2 = slice(9, 13, 1)
slice3 = slice(0, 2, 1)
slice4 = slice(5, 7, 1) #
slice5 = slice(11, 15, 1)
slice6 = slice(4, 8, 1) #

In [3]:
%%timeit
results_df = calculate_jaccard_index([array1, array2, array3])

8.57 ms ± 91 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%%timeit
merged_slices = get_merged_slices(slices, threshold)

150 ns ± 1.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Modified version to use dictionary with keys being sub_label ids and values being slices. If merged, new id is a string that combines previous ids by dashes.

In [12]:
def calculate_jaccard_index(slice1, slice2):
    range1 = set(range(slice1.start, slice1.stop))
    range2 = set(range(slice2.start, slice2.stop))
    intersection = len(range1.intersection(range2))
    union = len(range1.union(range2))
    return intersection / union if union else 0

def merge_slices(slice1, slice2):
    new_start = min(slice1.start, slice2.start)
    new_stop = max(slice1.stop, slice2.stop)
    return slice(new_start, new_stop)

def can_merge(slice1, slice2, threshold):
    return calculate_jaccard_index(slice1, slice2) > threshold

def merge_slices_if_possible(slices_dict, threshold):
    merged_dict = {}
    keys_to_delete = []
    for id1, slice1 in slices_dict.items():
        for id2, slice2 in slices_dict.items():
            if id1 != id2 and id1 not in keys_to_delete and id2 not in keys_to_delete:
                if can_merge(slice1, slice2, threshold):
                    merged_slice = merge_slices(slice1, slice2)
                    new_id = f"{id1}-{id2}"  # Generate a new ID by concatenating the IDs of the merged slices
                    merged_dict[new_id] = merged_slice
                    keys_to_delete.extend([id1, id2])
                    break
        if id1 not in keys_to_delete:
            merged_dict[id1] = slice1  # Add unmerged slices as they are
    return merged_dict

def get_merged_slices(slices_dict, threshold):
    while True:
        new_merged_dict = merge_slices_if_possible(slices_dict, threshold)
        if new_merged_dict == slices_dict:  # No further merges possible
            break
        slices_dict = new_merged_dict
    return slices_dict

# Example usage
slices_dict = {
    1: slice(4, 7, 1),
    2: slice(9, 13, 1),
    3: slice(0, 2, 1),
    4: slice(5, 7, 1),
    5: slice(11, 15, 1),
    6: slice(4, 8, 1)
}
threshold = 0.5
merged_slices_dict = get_merged_slices(slices_dict, threshold)
print(merged_slices_dict)

{'1-4-6': slice(4, 8, None), 2: slice(9, 13, 1), 3: slice(0, 2, 1), 5: slice(11, 15, 1)}
